In [ ]:
k = 10

In [ ]:
from streamsight.datasets import AmazonMovieDataset
from streamsight.settings import SlidingWindowSetting

dataset = AmazonMovieDataset()
data = dataset.load()
setting_sliding = SlidingWindowSetting(
    background_t=1406851200,
    window_size=60 * 60 * 24 * 300, # day times N
    n_seq_data=3,
    top_K=k
)
setting_sliding.split(data)


DEBUG    - streamsight.datasets.base - AmazonMovieDataset being initialized with 'data' as the base path.
DEBUG    - streamsight.datasets.base - AmazonMovieDataset is initialized.
INFO     - streamsight.datasets.base - AmazonMovieDataset is loading dataset...
DEBUG    - streamsight.datasets.base - Data file is in memory and in dir specified.
DEBUG    - streamsight.datasets.base - AmazonMovieDataset applying filters set.
DEBUG    - streamsight.preprocessing.preprocessor - 	interactions before preprocess: 8765568
DEBUG    - streamsight.preprocessing.preprocessor - 	items before preprocess: 182032
DEBUG    - streamsight.preprocessing.preprocessor - 	users before preprocess: 3826085
DEBUG    - streamsight.preprocessing.preprocessor - 	interactions after preprocess: 8765568
DEBUG    - streamsight.preprocessing.preprocessor - 	items after preprocess: 182032
DEBUG    - streamsight.preprocessing.preprocessor - 	users after preprocess: 3826085
INFO     - streamsight.datasets.base - AmazonMovieD

  0%|          | 0/5 [00:00<?, ?it/s]

DEBUG    - streamsight.settings.splitters - NPastInteractionTimestampSplitter(t=1406851200,t_lower=None,t_upper=25920000,n_seq_data=3) - Updating split point to t=1406851200
DEBUG    - streamsight.matrix.interaction_matrix - Performing lt(t, 1432771200)
DEBUG    - streamsight.matrix.interaction_matrix - Performing ge(t, 1406851200)
DEBUG    - streamsight.matrix.interaction_matrix - Performing get_user_n_last_interaction comparison
DEBUG    - streamsight.settings.splitters - NPastInteractionTimestampSplitter(t=1406851200,t_lower=None,t_upper=25920000,n_seq_data=3) - Split successful


 20%|██        | 1/5 [00:00<00:02,  1.66it/s]

DEBUG    - streamsight.settings.splitters - NPastInteractionTimestampSplitter(t=1406851200,t_lower=None,t_upper=25920000,n_seq_data=3) - Updating split point to t=1432771200
DEBUG    - streamsight.matrix.interaction_matrix - Performing lt(t, 1458691200)
DEBUG    - streamsight.matrix.interaction_matrix - Performing ge(t, 1432771200)
DEBUG    - streamsight.matrix.interaction_matrix - Performing get_user_n_last_interaction comparison
DEBUG    - streamsight.settings.splitters - NPastInteractionTimestampSplitter(t=1432771200,t_lower=None,t_upper=25920000,n_seq_data=3) - Split successful


...

DEBUG    - streamsight.settings.splitters - NPastInteractionTimestampSplitter(t=1484611200,t_lower=None,t_upper=25920000,n_seq_data=3) - Updating split point to t=1510531200
DEBUG    - streamsight.matrix.interaction_matrix - Performing lt(t, 1536451200)
DEBUG    - streamsight.matrix.interaction_matrix - Performing ge(t, 1510531200)
DEBUG    - streamsight.matrix.interaction_matrix - Performing get_user_n_last_interaction comparison
DEBUG    - streamsight.settings.splitters - NPastInteractionTimestampSplitter(t=1510531200,t_lower=None,t_upper=25920000,n_seq_data=3) - Split successful


100%|██████████| 5/5 [00:02<00:00,  1.95it/s]

DEBUG    - streamsight.settings.splitters - NPastInteractionTimestampSplitter(t=1510531200,t_lower=None,t_upper=25920000,n_seq_data=3) - Updating split point to t=1536451200
DEBUG    - streamsight.matrix.interaction_matrix - Performing lt(t, 1562371200)
DEBUG    - streamsight.matrix.interaction_matrix - Performing ge(t, 1536451200)
DEBUG    - streamsight.matrix.interaction_matrix - Performing get_user_n_last_interaction comparison
DEBUG    - streamsight.settings.splitters - NPastInteractionTimestampSplitter(t=1536451200,t_lower=None,t_upper=25920000,n_seq_data=3) - Split successful


6it [00:03,  1.95it/s]                       

INFO     - streamsight.settings.sliding_window_setting - Finished split with window size 25920000 seconds. Number of splits: 6
DEBUG    - streamsight.settings.base - Checking split attribute and sizes.
DEBUG    - streamsight.settings.base - Checking split attributes.
DEBUG    - streamsight.settings.base - Split attributes are set.
DEBUG    - streamsight.settings.base - Checking size of split sets.
DEBUG    - streamsight.settings.base - Size of split sets are checked.


In [ ]:
from streamsight.evaluator.evaluator_builder import EvaluatorBuilder

builder = EvaluatorBuilder()
builder.add_algorithm("ItemKNNStatic", {"K": k})
builder.add_algorithm("ItemKNNRolling", {"K": k})
builder.add_algorithm("ItemKNNIncremental", {"K": k})

builder.add_metric("PrecisionK", K=k)
builder.add_metric("RecallK", K=k)
builder.add_setting(setting_sliding)
evaluator = builder.build()

evaluator.run()

INFO     - streamsight.evaluator.evaluator_pipeline - Phase 1: Preparing the evaluator...


/Users/ngtzekean/Github/Streamsight/.venv/lib/python3.12/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 4.56s


/Users/ngtzekean/Github/Streamsight/.venv/lib/python3.12/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 4.35s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 9.54e-07s


...

/Users/ngtzekean/Github/Streamsight/.venv/lib/python3.12/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 1.49s


/Users/ngtzekean/Github/Streamsight/.venv/lib/python3.12/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 1.5s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 0.302s
INFO     - streamsight.algorithms.base - Fitting ItemKNNIncremental complete - Took 7.72s


100%|██████████| 6/6 [02:18<00:00, 23.15s/it]


In [ ]:
evaluator.metric_results(level="micro")

score  num_user
Algorithm                Timestamp    Metric                           
ItemKNNStatic(K=10)      t=1406851200 PrecisionK_10  0.000007    785024
                                      RecallK_10     0.000030    785024
                         t=1432771200 PrecisionK_10  0.000014    342331
                                      RecallK_10     0.000058    342331
                         t=1458691200 PrecisionK_10  0.000010    260637
                                      RecallK_10     0.000044    260637
                         t=1484611200 PrecisionK_10  0.000025    126083
                                      RecallK_10     0.000118    126083
                         t=1510531200 PrecisionK_10  0.000009     66625
                                      RecallK_10     0.000064     66625
                         t=1536451200 PrecisionK_10  0.000000       377
                                      RecallK_10     0.000000       377
ItemKNNRolling(K=10)     t=1406851200 PrecisionK_10  0.000712    785024
                                      RecallK_10     0.003552    785024
                         t=1432771200 PrecisionK_10  0.001766    342331
                                      RecallK_10     0.009503    342331
                         t=1458691200 PrecisionK_10  0.002885    260637
                                      RecallK_10     0.016929    260637
                         t=1484611200 PrecisionK_10  0.002889    126083
                                      RecallK_10     0.019928    126083
                         t=1510531200 PrecisionK_10  0.001177     66625
                                      RecallK_10     0.006403     66625
                         t=1536451200 PrecisionK_10  0.000796       377
                                      RecallK_10     0.006631       377
ItemKNNIncremental(K=10) t=1406851200 PrecisionK_10  0.000712    785024
                                      RecallK_10     0.003552    785024
                         t=1432771200 PrecisionK_10  0.002119    342331
                                      RecallK_10     0.011268    342331
                         t=1458691200 PrecisionK_10  0.003758    260637
                                      RecallK_10     0.021564    260637
                         t=1484611200 PrecisionK_10  0.003744    126083
                                      RecallK_10     0.024861    126083
                         t=1510531200 PrecisionK_10  0.002298     66625
                                      RecallK_10     0.011937     66625
                         t=1536451200 PrecisionK_10  0.001857       377
                                      RecallK_10     0.015915       377

In [ ]:
evaluator.metric_results(level="macro")

DEBUG    - streamsight.metrics.precision - Precision compute complete - PrecisionK_10
DEBUG    - streamsight.metrics.recall - Recall compute complete - RecallK_10
DEBUG    - streamsight.metrics.precision - Precision compute complete - PrecisionK_10
DEBUG    - streamsight.metrics.recall - Recall compute complete - RecallK_10
DEBUG    - streamsight.metrics.precision - Precision compute complete - PrecisionK_10
DEBUG    - streamsight.metrics.recall - Recall compute complete - RecallK_10


score
Algorithm                Metric                 
ItemKNNStatic(K=10)      PrecisionK_10  0.000011
                         RecallK_10     0.000047
ItemKNNRolling(K=10)     PrecisionK_10  0.001492
                         RecallK_10     0.008473
ItemKNNIncremental(K=10) PrecisionK_10  0.001828
                         RecallK_10     0.010248